In [10]:
#Imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array
from time import time

In [11]:
# load data 
from keras.datasets import cifar10

cls_names = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

"""
X_train.shape: (50000, 3, 32, 32)
X_test.shape: (10000, 3, 32, 32)
y: 10 labels
"""
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = y_train.reshape((y_train.shape[0]))
y_test = y_test.reshape((y_test.shape[0]))

# rechape data (..,...)
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

# new train reshape 
X_train.shape


(50000, 3072)

In [12]:
# GcForeest
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/cifar10.json")  # layer = 1   k=10   Extree + DTree
gc = GCForest(config)

In [13]:
# number of class
len(np.unique(y_train))

10

In [ ]:
t0 = time()
# X_enc is the concatenated predict_proba result of each estimators of the last layer of the GCForest model
    # X_enc.shape =
    #   (n_datas, n_estimators * n_classes): If cascade is provided
    #   (n_datas, n_estimators * n_classes, dimX, dimY): If only finegrained part is provided
    # You can also pass X_test, y_test to fit_transform method, then the accracy on test data will be logged when training.
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    # WARNING: if you set gc.set_keep_model_in_mem(True), you would have to use
    # gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test) to evaluate your model. 

[ 2018-12-05 11:59:30,975][cascade_classifier.fit_transform] X_groups_train.shape=[(50000, 3072)],y_train.shape=(50000,),X_groups_test.shape=[(10000, 3072)],y_test.shape=(10000,)
[ 2018-12-05 11:59:31,097][cascade_classifier.fit_transform] group_dims=[3072]
[ 2018-12-05 11:59:31,098][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-12-05 11:59:31,099][cascade_classifier.fit_transform] group_ends=[3072]
[ 2018-12-05 11:59:31,100][cascade_classifier.fit_transform] X_train.shape=(50000, 3072),X_test.shape=(10000, 3072)
[ 2018-12-05 11:59:31,405][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(50000, 3072), X_cur_test.shape=(10000, 3072)
[ 2018-12-05 12:01:13,446][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=46.00%
[ 2018-12-05 12:03:09,500][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=46.06%
[ 2018-12-05 12:05:04,390][kfold_wrapper.log_eval_metrics] Accuracy

In [ ]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of GCForest = {:.6f} %".format(acc * 100))

In [ ]:
tt = time() - t0     
print (" Time ",format(round(tt,3))," seconds")
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

# Neural network

In [ ]:
# estimators 
# Evaluating the ANN
t0 = time()
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential # initialize neural network library
from keras.layers import Dense # build our layers library
def build_classifier():
    classifier = Sequential() # initialize neural network
    classifier.add(Dense(units = 1024, kernel_initializer = 'uniform', activation = 'relu', input_dim = X_train.shape[1]))
    classifier.add(Dense(units = 512, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, epochs = 50)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
mean = accuracies.mean()
variance = accuracies.std()
print("Accuracy mean: "+ str(mean))
print("Accuracy variance: "+ str(variance))

tt = time() - t0     
print (" Time ",format(round(tt,3))," seconds")


# DecisionTreeClassifier

In [5]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy of DecisionTreeClassifier = {:.6f} %".format(acc * 100))
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

Test Accuracy of DecisionTreeClassifier = 26.520000 %
[[340  74  96  59  61  58  42  50 161  59]
 [ 74 274  70  92  57  55  47  67 106 158]
 [100  50 214  98 149 120 102  74  42  51]
 [ 63  60 115 196 100 137 133  93  49  54]
 [ 62  38 166  80 227 118 116 101  40  52]
 [ 54  52 107 174  83 210 105  99  55  61]
 [ 45  39 127 129 147 102 271  66  31  43]
 [ 79  93  83  91 115 110  59 243  58  69]
 [153 108  55  47  41  45  31  44 387  89]
 [ 83 166  70  71  39  54  47  82  98 290]]


             precision    recall  f1-score   support

          0       0.32      0.34      0.33      1000
          1       0.29      0.27      0.28      1000
          2       0.19      0.21      0.20      1000
          3       0.19      0.20      0.19      1000
          4       0.22      0.23      0.22      1000
          5       0.21      0.21      0.21      1000
          6       0.28      0.27      0.28      1000
          7       0.26      0.24      0.25      1000
          8       0.38      0.39   

# RandomForestClassifier

In [6]:
t0 = time()
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy of RandomForestClassifier = {:.6f} %".format(acc * 100))
 

tt = time() - t0     
print (" Time ",format(round(tt,3))," seconds")
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

Test Accuracy of RandomForestClassifier = 46.980000 %
(' Time ', '309.923', ' seconds')
[[569  36  52  17  37  21  25  29 166  48]
 [ 32 550  21  38  23  31  33  31  67 174]
 [105  44 317  93 143  67 116  60  28  27]
 [ 53  42  70 279  83 184 133  61  21  74]
 [ 55  22 155  56 402  43 144  80  23  20]
 [ 39  28  82 164  81 390  76  89  22  29]
 [ 11  35  83  69 104  56 568  26  12  36]
 [ 51  47  47  58  93  82  56 451  19  96]
 [ 92  91  21  23  22  34   8  24 609  76]
 [ 46 160  17  36  20  21  24  39  74 563]]


             precision    recall  f1-score   support

          0       0.54      0.57      0.55      1000
          1       0.52      0.55      0.54      1000
          2       0.37      0.32      0.34      1000
          3       0.33      0.28      0.30      1000
          4       0.40      0.40      0.40      1000
          5       0.42      0.39      0.40      1000
          6       0.48      0.57      0.52      1000
          7       0.51      0.45      0.48      1000
 

# ExtraTreesClassifier

In [7]:
t0 = time()
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy of ExtraTreesClassifier = {:.6f} %".format(acc * 100))
 

tt = time() - t0     
print (" Time ",format(round(tt,3))," seconds")
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

Test Accuracy of ExtraTreesClassifier = 47.320000 %
(' Time ', '133.711', ' seconds')
[[558  29  56  13  32  25  19  28 182  58]
 [ 35 554  11  37  20  30  39  33  61 180]
 [102  42 341  71 146  67 123  55  23  30]
 [ 58  47  73 284  84 177 121  59  19  78]
 [ 48  18 165  56 388  42 147  80  30  26]
 [ 43  25  87 153  82 389  85  76  23  37]
 [ 11  23  82  73 127  50 561  26   7  40]
 [ 44  45  43  58 109  85  42 448  30  96]
 [ 87  78  17  30  17  27  12  17 644  71]
 [ 39 161  13  32  14  24  30  36  86 565]]


             precision    recall  f1-score   support

          0       0.54      0.56      0.55      1000
          1       0.54      0.55      0.55      1000
          2       0.38      0.34      0.36      1000
          3       0.35      0.28      0.31      1000
          4       0.38      0.39      0.38      1000
          5       0.42      0.39      0.41      1000
          6       0.48      0.56      0.51      1000
          7       0.52      0.45      0.48      1000
   

# AdaBoostClassifier

In [8]:
t0 = time()
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy of AdaBoostClassifier = {:.6f} %".format(acc * 100))
 

tt = time() - t0     
print (" Time ",format(round(tt,3))," seconds")
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

Test Accuracy of AdaBoostClassifier = 33.080000 %
(' Time ', '1338.971', ' seconds')
[[336  35  50  42  33  35  38  53 302  76]
 [ 61 323  32  35  33  47  65  34 130 240]
 [ 80  72 121  71 185  90 196  81  74  30]
 [ 68  74  66 126  81 139 256  79  51  60]
 [ 49  42  81  44 258  77 299  85  39  26]
 [ 80  49  48 103  89 246 168  90  89  38]
 [ 10  45  48  77 119  55 560  40  22  24]
 [ 79  46  48  56 114  91 114 335  33  84]
 [105  67  23  16  24  35  21  20 521 168]
 [ 69 134  22  26  22  16  60  57 112 482]]


             precision    recall  f1-score   support

          0       0.36      0.34      0.35      1000
          1       0.36      0.32      0.34      1000
          2       0.22      0.12      0.16      1000
          3       0.21      0.13      0.16      1000
          4       0.27      0.26      0.26      1000
          5       0.30      0.25      0.27      1000
          6       0.32      0.56      0.40      1000
          7       0.38      0.34      0.36      1000
    

# Naive_bayes  GaussianNB

In [9]:
t0 = time()
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(X_train, y_train)

y_pred= gnb.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy of GaussianNB = {:.6f} %".format(acc * 100)) 

tt = time() - t0     
print (" Time ",format(round(tt,3))," seconds")
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

Test Accuracy of GaussianNB = 29.760000 %
(' Time ', '6.382', ' seconds')
[[494  20  39  10  84  34  50   9 200  60]
 [141 166  24  31  66  72 192  19 121 168]
 [225  24  83  15 292  48 209  21  54  29]
 [163  36  54  76 151 129 262  26  34  69]
 [ 86   8  57  26 417  38 265  22  50  31]
 [156  17  55  51 167 264 159  36  57  38]
 [106   2  60  18 228  46 467  15  19  39]
 [134  24  36  41 228  94 102 131  72 138]
 [168  41  18  17  56  83  39   8 471  99]
 [144  67  17  20  48  32 101  23 141 407]]


             precision    recall  f1-score   support

          0       0.27      0.49      0.35      1000
          1       0.41      0.17      0.24      1000
          2       0.19      0.08      0.12      1000
          3       0.25      0.08      0.12      1000
          4       0.24      0.42      0.30      1000
          5       0.31      0.26      0.29      1000
          6       0.25      0.47      0.33      1000
          7       0.42      0.13      0.20      1000
          8    